**Pre-processing: Step-1**

1. Read .dcm files and save pixel array data

Requirment: GDCM package

In [2]:
# loading gdrive for using in Google Colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# installing GDCM

In [3]:
import sys
print(sys.version)
!pip3 install ideep4py

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]


In [4]:
!git clone --branch master https://github.com/HealthplusAI/python3-gdcm.git && cd python3-gdcm && sudo dpkg -i build_1-1_amd64.deb && sudo apt-get install -f

Cloning into 'python3-gdcm'...
remote: Enumerating objects: 45, done.
remote: Total 45 (delta 0), reused 0 (delta 0), pack-reused 45
Unpacking objects: 100% (45/45), done.
Selecting previously unselected package build.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack build_1-1_amd64.deb ...
Unpacking build (1-1) ...
Setting up build (1-1) ...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [5]:
!sudo cp /usr/local/lib/gdcm.py /usr/local/lib/python3.6/dist-packages/.
!sudo cp /usr/local/lib/gdcmswig.py /usr/local/lib/python3.6/dist-packages/.
!sudo cp /usr/local/lib/_gdcmswig.so /usr/local/lib/python3.6/dist-packages/.
!sudo cp /usr/local/lib/libgdcm* /usr/local/lib/python3.6/dist-packages/.
!ldconfig

/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [0]:
import gdcm

In [0]:
# Verify that we can access Google Drive from colab
!ls "/content/drive/My Drive/"

In [0]:
# Define the base path
path_base = "/content/drive/My Drive/CovidCTNet/"

In [9]:
# !pip install pillow
!pip install pydicom

     |████████████████████████████████| 35.3MB 90kB/s 


In [0]:
import os
import sys
import pydicom
import numpy as np

In [0]:
sys.path.append(path_base + '/preprocessing')

In [0]:
from utilities import check_paths_validity, build_patient_list
from dcm_utilities import load_ct_scan, get_pixels_hu

In [43]:
dir_ct_scans = path_base + '/Data/DCM/'

dir_covid = dir_ct_scans + 'Covid/'
dir_hlthy = dir_ct_scans + 'Control/'
dir_pneum = dir_ct_scans + 'CAP/'
dir_test = dir_ct_scans + 'TEST/'

dir_out_ct_pixels_train = path_base + '/preprocessed/ct-pixels_train/'
dir_out_ct_pixels_test = path_base + '/preprocessed/ct-pixels_test/'

path_list = [path_base, dir_ct_scans,dir_covid, dir_hlthy, dir_pneum, dir_test, dir_out_ct_pixels_train,dir_out_ct_pixels_test]

# Verify all paths
check_paths_validity(path_list)

/content/drive/My Drive/CovidCTNet/  --> OK
/content/drive/My Drive/CovidCTNet//Data/DCM/  --> OK
/content/drive/My Drive/CovidCTNet//Data/DCM/Covid/  --> OK
/content/drive/My Drive/CovidCTNet//Data/DCM/Control/  --> OK
/content/drive/My Drive/CovidCTNet//Data/DCM/CAP/  --> OK
/content/drive/My Drive/CovidCTNet//Data/DCM/TEST/  --> OK
/content/drive/My Drive/CovidCTNet//preprocessed/ct-pixels_train/  --> OK
/content/drive/My Drive/CovidCTNet//preprocessed/ct-pixels_test/  --> OK


In [0]:
from pydicom import dcmread
from pydicom.pixel_data_handlers import gdcm_handler, pillow_handler

In [0]:
def extract_ct_pixels(lst_patients, input_dir, output_dir, prefix="E", overwrite=False,subfolder=''):
        
    print("*Extracting pixel array data. Input Dir:", input_dir)
  
    num_patients = 0
    for p in sorted(lst_patients[:]):
        num_patients += 1
        print("\n****************************************************************")
        print("{} / {} : <{}>".format(num_patients, str(len(lst_patients)), p))
        print("****************************************************************")

        out_file_ct_pixel = output_dir + prefix + str(p) + "_ct-pixels.npy"
        out_file_ct_orig_shape = output_dir + prefix + str(p) + "_ct-orig-shape.npy"
        out_file_ct_spacing = output_dir + prefix + str(p) + "_ct-spacing.npy"

        if (os.path.isfile(out_file_ct_pixel)==False or os.path.isfile(out_file_ct_orig_shape)==False or os.path.isfile(out_file_ct_spacing)==False or overwrite==True):
            try:
                patient_ct_slices, patient_ct_spacing = load_ct_scan(input_dir + p +subfolder)
                print("Slices(count):", len(patient_ct_slices))
                print("Spacing      :", patient_ct_spacing)
            except Exception as e:
                print("Error! @Function call: load_ct_scan -->", str(e))
                continue

            try:
                patient_ct_pixels = get_pixels_hu(patient_ct_slices)
                print("ct-pixels(shape):", patient_ct_pixels.shape)
            except Exception as e:
                print("Error! @Function call: get_pixels_hu -->", str(e))
                continue

            assert patient_ct_pixels.dtype == 'int16', print("patient_ct_pixels must be of type int16 instead of ",
                                                            patient_ct_pixels.dtype)

            np.save(out_file_ct_pixel, patient_ct_pixels)
            print("Saved file: ", out_file_ct_pixel)
            np.save(out_file_ct_orig_shape, patient_ct_pixels.shape)
            print("Saved file: ", out_file_ct_orig_shape)

            np.save(out_file_ct_spacing, patient_ct_spacing)
            print("Saved file: ", out_file_ct_spacing)

        else:
            print("Skipped: Output files already exist. Set overwrite = True to force regenerate outputs")
    
    print("\n*Finished")

In [28]:
# Build the list of all patients of each category
# if you have a subfolder in the folder of a patient, please add '/<subfolder name>' as input of "build_patient_list"

lst_covid_patients = build_patient_list(dir_covid,subfolder='/SR_3')
lst_covid_patients.sort()

lst_hlthy_patients = build_patient_list(dir_hlthy)
lst_hlthy_patients.sort()

lst_pneum_patients = build_patient_list(dir_pneum)
lst_pneum_patients.sort()

lst_test_patients = build_patient_list(dir_test) 
lst_test_patients.sort()

build_patient_list|Info: patient </content/drive/My Drive/CovidCTNet//Data/DCM/Covid/1641392+> found with 48 dcm files
build_patient_list|Info: patient </content/drive/My Drive/CovidCTNet//Data/DCM/Covid/1641266+> found with 60 dcm files
Patients detected:2
build_patient_list|Info: patient </content/drive/My Drive/CovidCTNet//Data/DCM/Control/PATIENT 2 (5)> found with 62 dcm files
build_patient_list|Info: patient </content/drive/My Drive/CovidCTNet//Data/DCM/Control/PATIENT 2 (4)> found with 66 dcm files
Patients detected:2
build_patient_list|Info: patient </content/drive/My Drive/CovidCTNet//Data/DCM/CAP/Patient_23> found with 31 dcm files
build_patient_list|Info: patient </content/drive/My Drive/CovidCTNet//Data/DCM/CAP/Patient_29> found with 40 dcm files
Patients detected:2
build_patient_list|Info: patient </content/drive/My Drive/CovidCTNet//Data/DCM/TEST/TEST (5)> found with 60 dcm files
build_patient_list|Info: patient </content/drive/My Drive/CovidCTNet//Data/DCM/TEST/TEST (1)> 

In [35]:
# Extract pixel data for Covid-PCR patients
extract_ct_pixels(lst_covid_patients, dir_covid, dir_out_ct_pixels_train, prefix='CPCR_',subfolder='/SR_3')

*Extracting pixel array data. Input Dir: /content/drive/My Drive/CovidCTNet//Data/DCM/Covid/

****************************************************************
1 / 2 : <1641266+>
****************************************************************
load_ct_scan|Info ==> loaded 60 slices from: /content/drive/My Drive/CovidCTNet//Data/DCM/Covid/1641266+/SR_3
Slices(count): 60
Spacing      : [5.   0.64 0.64]
ct-pixels(shape): (60, 512, 512)
Saved file:  /content/drive/My Drive/CovidCTNet//preprocessed/ct-pixels_train/CPCR_1641266+_ct-pixels.npy
Saved file:  /content/drive/My Drive/CovidCTNet//preprocessed/ct-pixels_train/CPCR_1641266+_ct-orig-shape.npy
Saved file:  /content/drive/My Drive/CovidCTNet//preprocessed/ct-pixels_train/CPCR_1641266+_ct-spacing.npy

****************************************************************
2 / 2 : <1641392+>
****************************************************************
load_ct_scan|Info ==> loaded 48 slices from: /content/drive/My Drive/CovidCTNet//Data/DCM/C

In [30]:
# Extract pixel data for Healthy patients
extract_ct_pixels(lst_hlthy_patients, dir_hlthy, dir_out_ct_pixels_train, prefix='H_')

*Extracting pixel array data. Input Dir: /content/drive/My Drive/CovidCTNet//Data/DCM/Control/

****************************************************************
1 / 2 : <PATIENT 2 (4)>
****************************************************************
load_ct_scan|Info ==> loaded 66 slices from: /content/drive/My Drive/CovidCTNet//Data/DCM/Control/PATIENT 2 (4)
Slices(count): 66
Spacing      : [5.   0.74 0.74]
ct-pixels(shape): (66, 512, 512)
Saved file:  /content/drive/My Drive/CovidCTNet//preprocessed/ct-pixels_train/H_PATIENT 2 (4)_ct-pixels.npy
Saved file:  /content/drive/My Drive/CovidCTNet//preprocessed/ct-pixels_train/H_PATIENT 2 (4)_ct-orig-shape.npy
Saved file:  /content/drive/My Drive/CovidCTNet//preprocessed/ct-pixels_train/H_PATIENT 2 (4)_ct-spacing.npy

****************************************************************
2 / 2 : <PATIENT 2 (5)>
****************************************************************
load_ct_scan|Info ==> loaded 62 slices from: /content/drive/My Drive/Co

In [31]:
# Extract pixel data for Pneumonia patients
extract_ct_pixels(lst_pneum_patients, dir_pneum, dir_out_ct_pixels_train, prefix='P_')

*Extracting pixel array data. Input Dir: /content/drive/My Drive/CovidCTNet//Data/DCM/CAP/

****************************************************************
1 / 2 : <Patient_23>
****************************************************************
load_ct_scan|Info ==> loaded 31 slices from: /content/drive/My Drive/CovidCTNet//Data/DCM/CAP/Patient_23
Slices(count): 31
Spacing      : [10.    0.75  0.75]
ct-pixels(shape): (31, 512, 512)
Saved file:  /content/drive/My Drive/CovidCTNet//preprocessed/ct-pixels_train/P_Patient_23_ct-pixels.npy
Saved file:  /content/drive/My Drive/CovidCTNet//preprocessed/ct-pixels_train/P_Patient_23_ct-orig-shape.npy
Saved file:  /content/drive/My Drive/CovidCTNet//preprocessed/ct-pixels_train/P_Patient_23_ct-spacing.npy

****************************************************************
2 / 2 : <Patient_29>
****************************************************************
load_ct_scan|Info ==> loaded 40 slices from: /content/drive/My Drive/CovidCTNet//Data/DCM/CAP/

In [44]:
# Extract pixel data for Test patients
extract_ct_pixels(lst_test_patients, dir_test, dir_out_ct_pixels_test, prefix='T_')

*Extracting pixel array data. Input Dir: /content/drive/My Drive/CovidCTNet//Data/DCM/TEST/

****************************************************************
1 / 2 : <TEST (1)>
****************************************************************
load_ct_scan|Info ==> loaded 341 slices from: /content/drive/My Drive/CovidCTNet//Data/DCM/TEST/TEST (1)
Slices(count): 341
Spacing      : [1.   0.75 0.75]
ct-pixels(shape): (341, 512, 512)
Saved file:  /content/drive/My Drive/CovidCTNet//preprocessed/ct-pixels_test/T_TEST (1)_ct-pixels.npy
Saved file:  /content/drive/My Drive/CovidCTNet//preprocessed/ct-pixels_test/T_TEST (1)_ct-orig-shape.npy
Saved file:  /content/drive/My Drive/CovidCTNet//preprocessed/ct-pixels_test/T_TEST (1)_ct-spacing.npy

****************************************************************
2 / 2 : <TEST (5)>
****************************************************************
load_ct_scan|Info ==> loaded 60 slices from: /content/drive/My Drive/CovidCTNet//Data/DCM/TEST/TEST (5)
Sli